# Price Predicition Model

This file creates a model using machine learning to predict listing prices in each neighborhood and tract.

[some inspiration](https://towardsdatascience.com/airbnb-price-prediction-using-linear-regression-scikit-learn-and-statsmodels-6e1fc2bd51a6)

In [1]:
# imports



In [3]:
# load datasets



In [4]:
# you know the rest...